<a href="https://colab.research.google.com/github/lakshay-sharma-2024/Assignment-1/blob/main/LSTM_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.utils import to_categorical

In [ ]:
df = pd.read_csv('cleaned_data.csv')
df.head()

,target,ids,date,flag,user,text,cleaned_text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",awww bummer shoulda get david carr third day
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset update facebook texting might cry result...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,dive many time ball manage save rest go bound
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole body feel itchy like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",behave mad see


In [ ]:
df.dropna(inplace=True)

In [ ]:
df.isna().sum()

target          0
ids             0
date            0
flag            0
user            0
text            0
cleaned_text    0
dtype: int64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df.cleaned_text,
    df.target,
    stratify=df.target,
    test_size=0.2,
    random_state=2)

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train,dtype=np.float32)
y_test = np.array(y_test,dtype=np.float32)

In [ ]:
batch_size = 5
max_len = len(df['cleaned_text'].max())
embedding_dim = 5
max_words = 10000
lstm_units = 5

In [ ]:
max_len

57

In [ ]:
df['target'].value_counts()

target
0    795793
4    795425
Name: count, dtype: int64

In [ ]:
tokenizer = Tokenizer(num_words=max_words)

tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_seq, maxlen=max_len)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_len)

y_train = np.array(y_train, dtype=np.float32)
y_test = np.array(y_test, dtype=np.float32)

In [ ]:
chunk_size = len(X_train_padded) // 10

X_train_chunks = np.array_split(X_train_padded, 10)
y_train_chunks = np.array_split(y_train, 10)


In [ ]:
def LSTM_Model(words,len,dim,units,num_labels):
  model = tf.keras.Sequential()
  model.add(Embedding(input_dim=words, output_dim=dim, input_length=len))
  model.add(LSTM(units=units, dropout=0.3, recurrent_dropout=0.3))
  model.add(Dense(num_labels, activation='softmax'))

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model

In [ ]:
model = LSTM_Model(words=10000, len=max_len, dim=10, units=8, num_labels=5)

for i, (X_chunk, y_chunk) in enumerate(zip(X_train_chunks, y_train_chunks)):
    print(f"Training on chunk {i+1}/{len(X_train_chunks)}")

    model.fit(X_chunk, y_chunk, epochs=5, batch_size=4, validation_split=0.2)



c:\Users\laksh\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Training on chunk 1/10
Epoch 1/5
25460/25460 ━━━━━━━━━━━━━━━━━━━━ 312s 12ms/step - accuracy: 0.6469 - loss: 0.6271 - val_accuracy: 0.7557 - val_loss: 0.5047
Epoch 2/5
25460/25460 ━━━━━━━━━━━━━━━━━━━━ 340s 13ms/step - accuracy: 0.7669 - loss: 0.4841 - val_accuracy: 0.7604 - val_loss: 0.4984
Epoch 3/5
25460/25460 ━━━━━━━━━━━━━━━━━━━━ 335s 13ms/step - accuracy: 0.7822 - loss: 0.4629 - val_accuracy: 0.7613 - val_loss: 0.4972
Epoch 4/5
25460/25460 ━━━━━━━━━━━━━━━━━━━━ 291s 11ms/step - accuracy: 0.7913 - loss: 0.4487 - val_accuracy: 0.7570 - val_loss: 0.4984
Epoch 5/5
25460/25460 ━━━━━━━━━━━━━━━━━━━━ 379s 15ms/step - accuracy: 0.7929 - loss: 0.4435 - val_accuracy: 0.7549 - val_loss: 0.5014
Training on chunk 2/10
Epoch 1/5
25460/25460 ━━━━━━━━━━━━━━━━━━━━ 409s 16ms/step - accuracy: 0.7579 - loss: 0.4972 - val_accuracy: 0.7609 - val_loss: 0.4932
Epoch 2/5
25460/25460 ━━━━━━━━━━━━━━━━━━━━ 331s 13ms/step - accuracy: 0.7757 - loss: 0.4728 - val_accuracy: 0.7628 - val_loss: 0.4880
Epoch 3/5
25460/

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test_padded, y_test, batch_size=10)
print(f"Test Accuracy: {test_accuracy}")


31825/31825 ━━━━━━━━━━━━━━━━━━━━ 73s 2ms/step - accuracy: 0.7708 - loss: 0.4759
Test Accuracy: 0.770628809928894


In [ ]:
model = LSTM_Model(words=10000, len=max_len, dim=5, units=10, num_labels=5)

for i, (X_chunk, y_chunk) in enumerate(zip(X_train_chunks, y_train_chunks)):
    print(f"Training on chunk {i+1}/{len(X_train_chunks)}")

    model.fit(X_chunk, y_chunk, epochs=2, batch_size=5, validation_split=0.3)



Training on chunk 1/10
Epoch 1/2
17822/17822 ━━━━━━━━━━━━━━━━━━━━ 162s 9ms/step - accuracy: 0.6164 - loss: 0.6584 - val_accuracy: 0.7508 - val_loss: 0.5097
Epoch 2/2
17822/17822 ━━━━━━━━━━━━━━━━━━━━ 173s 10ms/step - accuracy: 0.7630 - loss: 0.4941 - val_accuracy: 0.7518 - val_loss: 0.5048
Training on chunk 2/10
Epoch 1/2
17822/17822 ━━━━━━━━━━━━━━━━━━━━ 176s 10ms/step - accuracy: 0.7474 - loss: 0.5134 - val_accuracy: 0.7629 - val_loss: 0.4888
Epoch 2/2
17822/17822 ━━━━━━━━━━━━━━━━━━━━ 156s 9ms/step - accuracy: 0.7681 - loss: 0.4836 - val_accuracy: 0.7617 - val_loss: 0.4898
Training on chunk 3/10
Epoch 1/2
17822/17822 ━━━━━━━━━━━━━━━━━━━━ 151s 8ms/step - accuracy: 0.7593 - loss: 0.4985 - val_accuracy: 0.7642 - val_loss: 0.4870
Epoch 2/2
17822/17822 ━━━━━━━━━━━━━━━━━━━━ 169s 9ms/step - accuracy: 0.7683 - loss: 0.4833 - val_accuracy: 0.7646 - val_loss: 0.4882
Training on chunk 4/10
Epoch 1/2
17822/17822 ━━━━━━━━━━━━━━━━━━━━ 155s 9ms/step - accuracy: 0.7597 - loss: 0.4931 - val_accuracy: 0

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test_padded, y_test, batch_size=10)
print(f"Test Accuracy: {test_accuracy}")


31825/31825 ━━━━━━━━━━━━━━━━━━━━ 56s 2ms/step - accuracy: 0.7696 - loss: 0.4772
Test Accuracy: 0.7696012854576111


In [ ]:
def LSTM_Model_sig(words,len,dim,units,num_labels):
  model = tf.keras.Sequential()
  model.add(Embedding(input_dim=words, output_dim=dim, input_length=len))
  model.add(LSTM(units=units, dropout=0.2, recurrent_dropout=0.2))
  model.add(Dense(num_labels, activation='sigmoid'))

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model

In [ ]:
model = LSTM_Model_sig(words=10000, len=max_len, dim=10, units=10, num_labels=10)

for i, (X_chunk, y_chunk) in enumerate(zip(X_train_chunks, y_train_chunks)):
    print(f"Training on chunk {i+1}/{len(X_train_chunks)}")

    model.fit(X_chunk, y_chunk, epochs=2, batch_size=10, validation_split=0.3)



Training on chunk 1/10
Epoch 1/2


/home/lakshay/miniconda3/envs/rapids-env/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


8911/8911 ━━━━━━━━━━━━━━━━━━━━ 78s 9ms/step - accuracy: 0.6086 - loss: 0.7258 - val_accuracy: 0.7477 - val_loss: 0.5091
Epoch 2/2
8911/8911 ━━━━━━━━━━━━━━━━━━━━ 78s 9ms/step - accuracy: 0.7667 - loss: 0.4840 - val_accuracy: 0.7516 - val_loss: 0.5063
Training on chunk 2/10
Epoch 1/2
8911/8911 ━━━━━━━━━━━━━━━━━━━━ 75s 8ms/step - accuracy: 0.7574 - loss: 0.4977 - val_accuracy: 0.7620 - val_loss: 0.4897
Epoch 2/2
8911/8911 ━━━━━━━━━━━━━━━━━━━━ 77s 9ms/step - accuracy: 0.7753 - loss: 0.4694 - val_accuracy: 0.7620 - val_loss: 0.4885
Training on chunk 3/10
Epoch 1/2
8911/8911 ━━━━━━━━━━━━━━━━━━━━ 89s 10ms/step - accuracy: 0.7606 - loss: 0.4933 - val_accuracy: 0.7666 - val_loss: 0.4849
Epoch 2/2
8911/8911 ━━━━━━━━━━━━━━━━━━━━ 84s 9ms/step - accuracy: 0.7794 - loss: 0.4648 - val_accuracy: 0.7636 - val_loss: 0.4906
Training on chunk 4/10
Epoch 1/2
8911/8911 ━━━━━━━━━━━━━━━━━━━━ 89s 10ms/step - accuracy: 0.7600 - loss: 0.4888 - val_accuracy: 0.7660 - val_loss: 0.4826
Epoch 2/2
8911/8911 ━━━━━━━━━

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test_padded, y_test, batch_size=10)
print(f"Test Accuracy: {test_accuracy}")


31825/31825 ━━━━━━━━━━━━━━━━━━━━ 53s 2ms/step - accuracy: 0.7722 - loss: 0.4718
Test Accuracy: 0.7717160582542419


In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  0


In [1]:
!jupyter nbconvert --to PDF "drive/My drive/Colab Notebooks/LSTM.ipynb

/bin/bash: -c: line 1: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 2: syntax error: unexpected end of file
